In [1]:
import pygame
import random
import math
import numpy as np
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.8.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
SAFETY_DISTANCE = 20

In [24]:
class Car:
    def __init__(self, next_stop = None, current_traj = None): #pos input is a tuple
        self.pos = None
        self.direction = None
        self.distance = None
        self.next = None
        self.next_stop = next_stop
        self.current_traj = current_traj
        
        self.speed = 2
        self.max_speed = 4
        self.acceleration = 0.02
        self.deceleration = 0
        self.lookahead = self.speed * 20

        
        self.color = (255, 255, 255)
        
    def determine_distance(self):
        if self.next_stop.red == True:
            return min(math.dist(self.pos, self.next.pos), 
                       math.dist(self.pos, self.next_stop.pos))
        else:
            return math.dist(self.pos, self.next.pos)
    
    def step(self):
        #new stoplight
        if np.sum(np.sign(self.next_stop.pos - self.pos) + np.sign(self.direction)) == 0:
            self.next_stop = self.next_stop.next
            
        new_distance = self.determine_distance()

        if new_distance < self.lookahead + SAFETY_DISTANCE:
            self.speed = max(self.speed/2, 0)
        else:
            self.speed = min(self.speed+self.acceleration, self.max_speed)
        
        self.distance = new_distance
        self.lookahead = self.speed * 20
        self.pos = self.pos + (self.speed*self.direction)

In [17]:
class Stoplight:
    def __init__(self, pos, red = True): #pos input is a tuple
        self.pos = np.array(pos)
        self.next = None
        self.red = red
        self.color = (255, 0, 0)
        
    def step(self):
        self.red = not self.red
        self.change_color()
        
    def change_color(self):
        if self.red == True:
            self.color = (255, 0, 0)
        else:
            self.color = (0, 255, 0)

In [18]:
class Trajectory:
    def __init__(self, start, end): #pos input is a tuple
        self.start = np.array(start)
        self.end = np.array(end)
        self.next = None
        self.direction = self.get_direction()
        
    def get_direction(self):
        direction_not_norm = self.end - self.start
        return direction_not_norm/np.sqrt(np.sum((direction_not_norm)**2))

In [19]:
#at first we need to create the stoplight linked list
class Stop_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
    
    def append(self, other):
        if self.head == None:
            self.head = other
            return
        a = self.head
        while a.next != None:
            a = a.next
        a.next = other

In [20]:
#creating the trajectory linked list
class Traj_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
        
    def append(self, other):
        if self.head == None:
            self.head = other
            return
        a = self.head
        while a.next != None:
            a = a.next
        a.next = other

In [21]:
#lastly the car linked list
class Car_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
        
    def front_append(self, other):
        #appending the car
        if self.head == None:
            self.head = other
            self.head.next = Stoplight((0, 100))
            #initializing the other details of the car
            other.pos = other.current_traj.start
            other.direction = other.current_traj.direction
            other.distance = other.determine_distance()
            return
        
        a = self.head
        other.next = a
        self.head = other
        #initializing the other details of the car
        other.pos = other.current_traj.start
        other.direction = other.current_traj.direction
        other.distance = other.determine_distance()

In [26]:
time.sleep(6)
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Traffic Simulation")
img = pygame.image.load("intersection.png").convert()


sl = Stop_list()
tl = Traj_list()
cl = Car_list()
sl.append(Stoplight((400, 300)))
sl.append(Stoplight((400, 100)))
tl.append(Trajectory((400, 600), (400, 100)))

i = 0
while i < 3000:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    screen.fill((100, 100, 100))
    
    #stoplights
    if i%200 == 0:
        sl.head.step()
    stoplight = sl.head
    while stoplight.next != None:
        pygame.draw.line(screen, stoplight.color, stoplight.pos+np.array([-4, 0]), stoplight.pos+np.array([4, 0]), width = 4)
        stoplight = stoplight.next
    
    #cars
    if min(np.random.poisson(0.05), 1) == 1: #rate at which to spawn cars
        cl.front_append(Car(sl.head, tl.head))
    if cl.head != None:
        car = cl.head
        while car.next != None:
            car.step()
            pygame.draw.circle(screen, car.color, car.pos, 5)
            car = car.next
            
    pygame.display.update()
    i += 1

In [27]:
pygame.quit()